In [1]:
import pandas as pd

## Importing the CSVs


In [2]:
registration = pd.read_csv("./data/enc.csv")
payments = pd.read_csv("./data/pop.csv")

## Cleaning the data


In [3]:
registration.loc[registration["Are you subscribed to ENC Wits?"] == "Yes", [
    "Name and Surname", "Please select one of the payment options below"]]
registration['Name and Surname'] = registration['Name and Surname'].str.strip()
registration['Name and Surname'] = registration['Name and Surname'].str.title()
registration = registration.drop_duplicates(subset="Name and Surname") # dropping duplicates
payments['Name + Surname'] = payments['Name + Surname'].str.strip()
payments['Name + Surname'] = payments['Name + Surname'].str.title()

# registration[registration.duplicated(subset="Name and Surname")] // duplicated data

## Merge of proof of payment and the registration to show payment and payment plan progress


In [4]:
payment_process = pd.merge(
    registration, payments, left_on='Name and Surname', right_on='Name + Surname', how="left")

inner_joint_data = pd.merge(registration, payments, left_on='Name and Surname',
               right_on='Name + Surname', how="inner") # inner join that shows only the common data between registration and payments.


# Cleaning the data

- Clean the amount paid so that it becomes 0 if it return Nan and turns every amount into an integer
- removing concatenated values in subscribed or not field


In [5]:
def clean_amount(x):
    if pd.isna(x):
        return 0  # or handle NaN values as needed
    return int(float(str(x).replace('R', '')))



payment_process["How much did you pay?"] = payment_process["How much did you pay?"].apply(
    clean_amount)

- Clean the data so that the amount is only in numeric not alphanumeric


## Merge Duplicate rows


In [6]:

grouped_payments = payment_process.groupby(
    "Name and Surname", as_index=False).sum()

grouped_payments["Are you subscribed to ENC Wits?"] = grouped_payments["Are you subscribed to ENC Wits?"].str.strip()
grouped_payments["Are you subscribed to ENC Wits?"] = grouped_payments["Are you subscribed to ENC Wits?"].replace({
                                                                                                                 "YesYes": "Yes", "NoNo": "No"})
# grouped_payments.loc[50:100]

## Display amount and **Name + Surname** of those who are subscribed


In [7]:
subscribed = grouped_payments.loc[grouped_payments["Are you subscribed to ENC Wits?"] == "Yes", [
    "Name and Surname", "How much did you pay?"]]
total_amount_subscribed = 225*(len(subscribed)-3)+3*180
# total_amount_subscribed

subscribed

,Name and Surname,How much did you pay?
1,Abongiwe Kentane,0
2,Anele Sontenze,0
5,Aphiwe Vezi,0
6,Asithandile Madikane,0
7,Ayanda Chiliza,0
...,...,...
112,Tlhokomelo Mathaba,0
113,Tshegofatso Setsepu,225
116,Tumi Motala,0
118,Zanele Lusenga,0


## Display the **Name + Surname** of those that are not subscribed


In [8]:

not_subscribed = grouped_payments.loc[grouped_payments["Are you subscribed to ENC Wits?"] == "No", [
    "Name and Surname", "How much did you pay?"]]
total_amount_unsubscribed = 300*len(not_subscribed)
# total_amount_unsubscribed

# Adding columns of the money left that needs to be paid


### Subscribed and paid before June


In [9]:

grouped_payments['Timestamp_y'] = pd.to_datetime(
    grouped_payments['Timestamp_y'], format='%d/%m/%Y %H:%M:%S', errors='coerce')

### Total amount paid currently


In [10]:
total = grouped_payments["How much did you pay?"].sum() + 80+ 225
print(total)
grouped_payments.loc[grouped_payments["How much did you pay?"] != 0,['Name and Surname', 'How much did you pay?']]

6060


,Name and Surname,How much did you pay?
11,Blessing Kodze,255
16,Davina Mabena,225
17,Dineo Lamola,100
21,Fezile Legodi,75
22,Hans Dladla,300
28,Keabetsoe Molefe,225
42,Lindokuhle Zama,100
44,Liyema Ndwandwa,75
49,Luyanda Mabena,75
50,Magdaline Maremeni,300


## Number of people registered.

In [11]:
len(grouped_payments)

121

## Expected total amount from sign ups

In [12]:
expected_amount_from_signups = total_amount_subscribed + total_amount_unsubscribed
print(expected_amount_from_signups)

30990


## Number of people who have completed their payments.

In [13]:
t1=len(subscribed.loc[subscribed["How much did you pay?"]==225,["Name and Surname", "How much did you pay?"]])
t2 =len(not_subscribed.loc[not_subscribed["How much did you pay?"]==300,["Name and Surname", "How much did you pay?"]])
t1+t2+1

18

## Percentage of the money that we have collected so far

In [14]:
percentage_completed=(total/expected_amount_from_signups)*100
print(percentage_completed)

19.554695062923525


## List of unsubscribed members and haven't finished paying.

## List of the subscribed members and money they still need to pay


## List of unsubscribed members and the money they still need to pay

## Number of people that have fully paid their amounts.

# People not registered on the signups

1. Liyema Ndwandwa
2. Ntombi Mphahlela
3. Khanyo Mtongana

In [24]:
print(f"Total number of people who signed up: {len(grouped_payments)}")
print(f"Total amount expected: {expected_amount_from_signups}")
print(f"Payments that we have right now in percentage is : {percentage_completed:.0f}%")


Total number of people who signed up: 121
Total amount expected: 30990
Payments that we have right now in percentage is : 20%


- Request access to the pop pdfs to check for duplicate payements
- We requested payment screenshots from Ps Neo and noticed there are people who paid but aren't signed up.